In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix
    )

from panda_station import *

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
parser = Parser(plant)

add_package_paths(parser)
print(parser.package_map().GetPackageNames())

panda_hand = parser.AddModelFromFile(find_resource("models/modified_panda_hand/sdf/welded_panda_hand.sdf"))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("panda_hand"))


plant.Finalize()
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=scene_graph.get_query_output_port(),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()


frame = plant.GetFrameByName("panda_hand", panda_hand)
plant_context = plant.GetMyContextFromRoot(diagram_context)
#plant.SetPositions(plant_context, panda_hand, [0.04, 0.04])
print(frame.CalcPoseInWorld(plant_context))

diagram.Publish(diagram_context)


['manipulation_station', 'modified_panda_hand', 'tables']
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)
